In [1]:
import numpy as np
import math
T=5
NX=4
NU=2
DT=0.2
WB=110
MAX_STEER = np.deg2rad(45.0)  # maximum steering angle [rad]
MAX_DSTEER = np.deg2rad(30.0)  # maximum steering speed [rad/s]
MAX_SPEED = 25#55.0 / 3.6  # maximum speed [m/s]
MIN_SPEED = -25#-20.0 / 3.6  # minimum speed [m/s]
MAX_ACCEL = 5.0  # maximum accel [m/ss]
class State:
     
    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.x = x
        self.y = y
        self.yaw = yaw
        self.v = v
        self.predelta = None
    def array(self):
        return np.array([self.x,self.y,self.v,self.yaw],dtype=np.float64)
class Motion():

    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.state=State(x=x, y=y, yaw=yaw, v=v)
        
    def get_state(self):
        return State(x=self.state.x, y=self.state.y, yaw=self.state.yaw, v=self.state.v)
        
    def set_state(self,state):
        self.state=State(x=state.x, y=state.y, yaw=state.yaw, v=state.v)
    def get_next_state(self, a, delta, curr_state=-1):
        if curr_state==-1:
            curr_state=self.state
        # input check
        if delta >= MAX_STEER:
            delta = MAX_STEER
        elif delta <= -MAX_STEER:
            delta = -MAX_STEER

        next_x = curr_state.x + curr_state.v * math.cos(curr_state.yaw) * DT
        next_y = curr_state.y + curr_state.v * math.sin(curr_state.yaw) * DT
        next_yaw = curr_state.yaw + curr_state.v / WB * math.tan(delta) * DT
        next_v = curr_state.v + a * DT
        #print(curr_state.v,a,DT,next_v)
        if next_v > MAX_SPEED:
            next_v = MAX_SPEED
        elif next_v < MIN_SPEED:
            next_v = MIN_SPEED

        return State(x=next_x,y=next_y,v=next_v,yaw=next_yaw)
    
    def _get_next_state(self, a, delta, curr_state=-1):
        if curr_state==-1:
            curr_state=self.state
        # input check
        if delta >= MAX_STEER:
            delta = MAX_STEER
        elif delta <= -MAX_STEER:
            delta = -MAX_STEER
            
        A, B, C = get_linear_model_matrix(curr_state.v, curr_state.yaw, delta)
        
        X=np.array([curr_state.x,curr_state.y,curr_state.v,curr_state.yaw],dtype=np.float64)
        U=np.array([a,delta],dtype=np.float64)
        [next_x,next_y,next_v,next_yaw]=np.matmul(A,X) + np.matmul(B,U)+ C
        
        
        if next_v > MAX_SPEED:
            next_v = MAX_SPEED
        elif next_v < MIN_SPEED:
            next_v = MIN_SPEED

        return State(x=next_x,y=next_y,v=next_v,yaw=next_yaw),[A,B,C]


    def get_trajectory(self,accels, deltas,time_step):
        xbar =np.zeros((NX, time_step + 1))
        xbar[:, 0] = self.state.x,self.state.y,self.state.v,self.state.yaw
        #_state=State(x=state.x,y=state.y,v=state.v,yaw=state.yaw)
        _state=Motion.get_state(self)
        for (accel, delta, i) in zip(accels, deltas, range(1, T + 1)):
            next_state = Motion._get_next_state(self,accel, delta, curr_state=_state)
            xbar[:, i] = next_state.x,next_state.y,next_state.v,next_state.yaw
            _state=next_state
        

        return xbar


    
def get_linear_model_matrix(v, phi, delta):
        A = np.zeros((NX, NX))
        A[0, 0] = 1.0
        A[1, 1] = 1.0
        A[2, 2] = 1.0
        A[3, 3] = 1.0
        A[0, 2] = DT * math.cos(phi)
        A[0, 3] = - DT * v * math.sin(phi)#
        A[1, 2] = DT * math.sin(phi)
        A[1, 3] = DT * v * math.cos(phi)#
        A[3, 2] = DT * math.tan(delta) / WB

        B = np.zeros((NX, NU))
        B[2, 0] = DT
        B[3, 1] = DT * v / (WB * math.cos(delta) ** 2)#

        C = np.zeros(NX)
        C[0] = DT * v * math.sin(phi) * phi
        C[1] = - DT * v * math.cos(phi) * phi
        C[3] = - DT * v * delta / (WB * math.cos(delta) ** 2)
        #C = np.zeros(NX)
        return A, B, C
    
def np_quad_form(a,b):
        res=a @ b @ a.T
        return res
    
def calc_cost( x, u,xref,t,_B):
        
        R = np.diag([0.01, 0.01])  # input cost matrix
        Rd = np.diag([0.01, 1.0])  # input difference cost matrix
        Q = np.diag([1.0, 1.0, 0.5, 50])  # state cost matrix
        Qf = Q  # state final matrix
        

        #cost += cvxpy.quad_form(u[:, t], R)
        #print(u,R)
        l = np_quad_form(u, R) + np_quad_form(xref - x, Q)
        
        
        #if t < (T - 1):
        #    l = np_quad_form(u[:, t + 1] - u[:, t], Rd)
            
        
        #l = np_quad_form(xref[:, T] - x[:, T], Qf)


        # compute derivatives of cost
        
        l_x1=2*Q[0,0]*(xref[0]-x[0])*(-1)
        l_x2=2*Q[1,1]*(xref[1]-x[1])*(-1)
        l_x3=2*Q[2,2]*(xref[2]-x[2])*(-1)
        l_x4=2*Q[3,3]*(xref[3]-x[3])*(-1)
            
        l_x=np.array([l_x1,l_x2,l_x3,l_x4])
        l_xx=2 * Q
        Qdiag=np.array([Q[0,0], Q[1,1], Q[2,2], Q[3,3]],dtype=np.float64)
        error=np.array(xref-x)
        #x=Ax+Bu+C
        #dx/du=d(B*u)=B
        l_u1 = 2. * u[0] * R[0,0] + np.sum(2*Qdiag*error*(-1)*_B[:,0])
        l_u2 = 2. * u[1] * R[1,1] + np.sum(2*Qdiag*error*(-1)*_B[:,1])
        
        l_u = np.array([l_u1, l_u2])

        l_uu = 2. * R
        
        l_ux = np.zeros((NU, NX))

        return l, l_x, l_xx, l_u, l_uu, l_ux

In [2]:
def sim_n_steps(x0,U,xref):
    X=np.zeros((NX,T+1),dtype=np.float64)
    X[:,0]=x0
    _MS=Motion()
    total_grad=0
    for t in range(T):
        _state=State(x=X[0,t],y=X[1,t],v=X[2,t],yaw=X[3,t])
        _MS.set_state(_state)
        nst,Const=_MS._get_next_state(U[0,t],U[1,t])
        B=Const[1]
        x_t1=[nst.x,nst.y,nst.v,nst.yaw]
        u_t=U[:,t]
        xref_t=xref[:,t]
        cc=calc_cost(x_t1,u_t,xref_t,t,B)
        du=0.2*cc[3]
        if t <T-1 : U[:,t+1]-=du
        X[:,t+1]=x_t1.copy()
        #print(X,cc[0],cc[3])
        total_grad+=du
    return total_grad,U.copy()
np.set_printoptions(suppress=True)
Test=False
if Test==True:
    _xref=np.ones((4,5+1),dtype=np.float64)
    _xref[0,:]=5
    _xref[1,:]=5
    _xref[2,:]=5
    _xref[3,:]=np.arctan2(5,5)
    _dref=np.zeros((5),dtype=np.float64)

    _x0=np.zeros((NX),dtype=np.float64)
    _x0[2]=1.0
    _x0[3]=np.arctan2(1,1)
    _u=np.zeros((NU,T),dtype=np.float64)
    _MS=Motion()
    ret=sim_n_steps(_x0,_u,_xref)
    print(ret[0],'\n\n',ret[1][0],'\n',ret[1][3])

In [3]:
from Map_Tracker import Map_Tracker
x=np.arange(0,300,5)
y=np.arange(0,60,1)
_map=np.array([x,y],dtype=np.float64).T
class ENV():
    def __init__(self):
        self._ms=Motion()
        self.map_tk=Map_Tracker(_map,qlen=15)
        
    def reset(self,x=0,y=0,v=0,yaw=0):
        self._ms.set_state(State(x=x,y=y,v=v,yaw=yaw))
        n_traj,_,_=self.map_tk.run(self._ms.state)
        return self._ms.get_state(),n_traj
    
    def step(self,accl,delta):
        _state=self._ms.get_next_state(accl,delta)
        n_traj,_,_=self.map_tk.run(_state)
        self._ms.set_state(_state)
        return self._ms.get_state(),n_traj

        
        

In [4]:
from Data_Processor import Data_Processor
env=ENV()
X,traj = env.reset()
U=np.zeros((NU,T),dtype=np.float64)
dp =Data_Processor(_speed=1,tlen=15)
#traj=dp.update_trajectory(traj.T)

_dt=0.2
_dl=1.0
_tstep=5

for i in range(100):
    
    dp.update_trajectory(traj)
    _xref,_=dp.get_ref_trajectory(X,_dl,_dt,_tstep)
    _,U=sim_n_steps(X.array(),U,_xref)
    #print(U)
    X,traj=env.step(U[0,-1],U[1,-1])
    print('..........................')
    print(X.array())
    print(_xref[:,0])

..........................
[0.         0.         0.00777867 0.        ]
[0.         0.         1.         0.19739556]
..........................
[0.00155573 0.         0.02305685 0.00000001]
[0.         0.         1.         0.19739556]
..........................
[0.0061671  0.         0.04550136 0.00000004]
[0.         0.         1.         0.19739556]
..........................
[0.01526738 0.         0.07472926 0.00000018]
[0.         0.         1.         0.19739556]
..........................
[0.03021323 0.         0.11031229 0.00000055]
[0.         0.         1.         0.19739556]
..........................
[0.05227569 0.00000002 0.15178164 0.00000139]
[0.         0.         1.         0.19739556]
..........................
[0.08263202 0.00000006 0.19863289 0.00000304]
[0.         0.         1.         0.19739556]
..........................
[0.1223586  0.00000018 0.2503311  0.000006  ]
[0.         0.         1.         0.19739556]
..........................
[0.17242482 0.0000004